In [2]:
import torch, json
from models.codec import SwinCrossScaleCodec

config = json.load(open("/Users/tracy/Desktop/swin-18k-scale-baseline/config.json", 'r'))
# model = SwinCrossScaleCodec(**config)
model = SwinCrossScaleCodec(patch_size = [3,2],
                 swin_depth = 2,
                 swin_heads = [3,3,6,12,24],
                 window_size = 4,
                 mlp_ratio = 4.,
                 in_dim = 2, 
                 in_freq = 192, 
                 h_dims = [45, 45, 72, 96, 192, 384], 
                 max_streams = 6, 
                 proj = [4,4,2,2,2,2], 
                 fuse_net = "window",
                 overlap = 2, 
                 num_vqs = 6, 
                 codebook_size = 1024, )
# model.load_state_dict(torch.load("/Users/tracy/Desktop/swin-18k-scale-baseline/best.pt", map_location="cpu")["model_state_dict"])

Apply Residual-Based Cross Attention Fusion Net for Swin Codec | Type: window
Quantization Vis: 
     Freq dims:  [2, 2, 4, 8, 16, 32]
     Channel(hidden) dims:  [384, 384, 192, 96, 72, 45]
     projections from:  [768, 768, 768, 768, 1152, 1440]
     projections to:  [192, 192, 384, 384, 576, 720]
     group_vq_dims:  [384, 384, 768, 768, 1152, 1440]


In [4]:
model.decoder

SwinCrossScaleDecoder(
  (patch_deembed): PatchDeEmbed(
    (de_proj1): Conv2d(45, 270, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (de_proj2): Conv2d(45, 2, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (blocks): ModuleList(
    (0): SwinTLayer(
      (swint_blocks): ModuleList(
        (0-1): 2 x SwinTBlock(
          (norm1): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
          (attn): WindowAttention(
            (qkv): Linear(in_features=384, out_features=1152, bias=True)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=384, out_features=384, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
            (softmax): Softmax(dim=-1)
          )
          (norm2): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
          (mlp): FeedForward(
            (linear_1): Linear(in_features=384, out_features=1536, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (line

In [5]:
import torchaudio
import torch
audio_path = "/Users/tracy/Desktop/Audio_Codec/swin-debug-vis/test/spanish_instance1.wav"
model.eval()
x, sr = torchaudio.load(audio_path)
x = x[:, :-80]

In [12]:
model.ft(x).shape

torch.Size([1, 192, 2000])

In [13]:
model.train()
outputs = model.train_one_step(x, None, streams=6)

model.eval()
with torch.inference_mode():
   test_outputs =  model.test_one_step(x, None, streams=6)

In [11]:
import torch.nn as nn
import torch

embed_dim = 6 
num_heads = 1

multihead_attn = nn.MultiheadAttention(
                                embed_dim=embed_dim,
                                num_heads=num_heads,
                                dropout=0.0,
                                bias=False,
                                batch_first=True)

x = torch.randn(1, 10, 6)

attn_output, attn_output_weights = multihead_attn(query=x, key=x, value=x)

In [19]:
in_freq = 192
patch_size = (3,2)
max_streams=6
[(in_freq//patch_size[0])//2**(max_streams-1)] + [(in_freq//patch_size[0])//2**i for i in range(max_streams-1, 0, -1)]

[2, 2, 4, 8, 16, 32]

In [13]:
from models.codec import SwinCrossScaleCodec
codec = SwinCrossScaleCodec(patch_size = [3,2],
                                swin_depth = 2,
                                swin_heads = [3,3,6,12,24],
                                window_size = 4,
                                mlp_ratio = 4.,
                                in_dim = 2, 
                                in_freq = 192, 
                                h_dims = [45, 45, 72, 96, 192, 384], 
                                max_streams = 6, 
                                proj = [4,4,2,2,2,2], 
                                overlap = 2, 
                                num_vqs = 6, 
                                codebook_size = 1024, 
                                fuse_net=True)
outputs = codec.train_one_step(x=torch.randn(1,47920),
                        x_feat=torch.randn(1,192,600,2),
                        streams=6)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Use Attn Fuse Merge Net for Swin Codec
Audio Codec 18.0kbps Initialized
Quantization Vis: 
     Freq dims:  [2, 2, 4, 8, 16, 32]
     Channel(hidden) dims:  [384, 384, 192, 96, 72, 45]
     projections from:  [768, 768, 768, 768, 1152, 1440]
     projections to:  [192, 192, 384, 384, 576, 720]
     group_vq_dims:  [384, 384, 768, 768, 1152, 1440]
enc shape torch.Size([1, 600, 384]) dec shape torch.Size([1, 600, 384])
pre_fuse 0 Sequential(
  (0): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=384, out_features=384, bias=False)
  )
)
Sequential(
  (0): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=384, out_features=384, bias=False)
  )
)
<bound method Sequential.forward of Sequential(
  (0): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=384, out_features=384, bias=False)
  )
)>


TypeError: forward() got an unexpected keyword argument 'query'

In [3]:
from models.swin import WindowAlignment

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
align = WindowAlignment(
    freq_size=2,
    d_model=24,
    num_heads=1,
    window_size=8,
    shift_size=4
)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [1]:
import torch
B, F, T, C = 1, 2, 10, 24

x = torch.randn(B, F*T, C)
y = torch.randn(B, F*T, C)

In [5]:
x, y = align(
    (x, y)
)
print(x.shape, y.shape)

torch.Size([1, 20, 24]) torch.Size([1, 20, 24])


In [6]:
shift_align = torch.nn.Sequential(
    *[WindowAlignment(
    freq_size=2,
    d_model=24,
    num_heads=1,
    window_size=8,
    shift_size=0
),WindowAlignment(
    freq_size=2,
    d_model=24,
    num_heads=1,
    window_size=8,
    shift_size=4
)]
)

In [8]:
x, y = shift_align((x,y))
print(x.shape, y.shape)

torch.Size([1, 20, 24]) torch.Size([1, 20, 24])


In [1]:
from models.codec import SwinCrossScaleCodec
import torch

model = SwinCrossScaleCodec(
                        patch_size = [3,2],
                        swin_depth = 2,
                        swin_heads = [3,3,6,12,24],
                        window_size = 4,
                        mlp_ratio = 4.,
                        in_dim = 2, 
                        in_freq = 192, 
                        h_dims = [45, 45, 72, 96, 192, 384], 
                        max_streams = 6, 
                        proj = [4,4,2,2,2,2], 
                        overlap = 2, 
                        num_vqs = 6, 
                        codebook_size = 1024, 
                        fuse_net="window",
                        is_causal=False,
)

Codec Causality: False
Apply Residual-Based Cross Attention Fusion Net for Swin Codec | Type: window
Quantization Vis: 
     Freq dims:  [2, 2, 4, 8, 16, 32]
     Channel(hidden) dims:  [384, 384, 192, 96, 72, 45]
     projections from:  [768, 768, 768, 768, 1152, 1440]
     projections to:  [192, 192, 384, 384, 576, 720]
     group_vq_dims:  [384, 384, 768, 768, 1152, 1440]


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [3]:
x = torch.randn(2, 47920)

In [5]:
output = model.test_one_step(x, None, streams=6)
recon_audio = output["recon_audio"]
recon_audio[0, 10:50]

tensor([ 7.4670e-03,  7.3609e-03,  1.5603e-02, -8.2264e-03, -8.8969e-03,
        -7.4742e-03,  6.9204e-03,  3.4534e-02, -9.7964e-03,  1.8173e-03,
        -1.9554e-02, -3.3295e-03, -4.7611e-03,  3.7022e-03, -3.4558e-03,
         1.7518e-04,  8.2543e-06,  5.7664e-03,  1.4516e-02, -6.8270e-04,
         5.9632e-04,  2.1606e-04,  2.2192e-02, -1.5404e-02, -9.3460e-03,
         6.1011e-03, -4.2672e-03, -9.6518e-03,  7.0921e-03, -4.6471e-03,
        -2.0705e-03, -1.2121e-03, -6.5024e-03,  5.7794e-03,  9.7892e-03,
         3.9240e-03,  1.3893e-03, -7.7010e-03, -5.9469e-05,  6.3737e-03])

In [4]:
multi_codes, enc_shape = model.encode(x, num_streams=6)
recon_audio_test = model.decode(multi_codes, enc_shape)
recon_audio_test[0, 10:50]

tensor([ 7.4670e-03,  7.3609e-03,  1.5603e-02, -8.2264e-03, -8.8969e-03,
        -7.4742e-03,  6.9204e-03,  3.4534e-02, -9.7964e-03,  1.8173e-03,
        -1.9554e-02, -3.3295e-03, -4.7611e-03,  3.7022e-03, -3.4558e-03,
         1.7518e-04,  8.2543e-06,  5.7664e-03,  1.4516e-02, -6.8270e-04,
         5.9632e-04,  2.1606e-04,  2.2192e-02, -1.5404e-02, -9.3460e-03,
         6.1011e-03, -4.2672e-03, -9.6518e-03,  7.0921e-03, -4.6471e-03,
        -2.0705e-03, -1.2121e-03, -6.5024e-03,  5.7794e-03,  9.7892e-03,
         3.9240e-03,  1.3893e-03, -7.7010e-03, -5.9469e-05,  6.3737e-03])

In [32]:
# Suppose `code_tensor` is your tensor of indices from the VQ-VAE model.
code_tensor = multi_codes[0][0] # This should be a tensor with the shape [batch_size, height, width] or [batch_size, sequence_length]

# Flatten the tensor to get a single list of indices.
flat_indices = code_tensor.flatten()

# Calculate the total number of indices used (which is also the total number of encodings generated by the model).
total_indices = flat_indices.shape[0]

# Count the frequency of each index.
index_counts = torch.bincount(flat_indices, minlength=1024)

# Calculate the probability of each index.
index_probabilities = index_counts.float() / total_indices

# Calculate entropy.
# Add a small value to probability to avoid log(0).
entropy = -(index_probabilities * torch.log(index_probabilities + 1e-10)).sum()

# Calculate the percentage of used entries.
# We only consider indices with at least one count.
used_entries = torch.sum(index_counts > 0).item()
percentage_used = used_entries / 1024 * 100

print(f'Entropy: {entropy.item()}')
print(f'Percentage of VQ entries used: {used_entries}/1024 = {percentage_used}%')

Entropy: 1.7460907697677612
Percentage of VQ entries used: 12/1024 = 1.171875%
